# Configuring IP over UDP

This example notebook shows how a user can configure the Ethernet connection of their board.

This example applies to all board models that have an Ethernet port.

## Naludaq Version

MinVersion: 0.33.2

In [ ]:
# Print Naludaq version 
import naludaq
print(f"Naludaq version: {naludaq.__version__}")

## Compatible Boards

* 

# Imports and Variables

In [ ]:
# Useful for debugging, but not normally necessary
%load_ext autoreload
%autoreload 2

In [ ]:
from naludaq.board import Board, startup_board
from naludaq.communication import ControlRegisters
from naludaq.controllers import get_connection_controller

import os

## Logging

The logger setup below will show additional information from the NaluDAQ package. This is useful for debugging purposes, but
may be skipped if not needed.

In [ ]:
import logging


def setup_logger(level=logging.INFO):
    """Setup a basic logger."""
    logger = logging.getLogger()
    handler = logging.StreamHandler()
    handler.setFormatter(
        logging.Formatter("%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s")
    )
    logger.addHandler(handler)
    logger.setLevel(level)
    suppress = [
        "naludaq.UART",
        "naludaq.FTDI",
    ]
    for name in suppress:
        logging.getLogger(name).setLevel(logging.CRITICAL)
    return logger


try:
    logger.debug("logger already setup")
except:
    logger = setup_logger()

## Board Configuration Options

The following parameters must be provided to successfully set the IP address:

- board_model:      : The board model. Valid options are listed below (as of 7/31/25):
                       
                       Options:
                       

- default_board_addr: (*ip_address*, *port*)
  - Refer to the Quick Start guide for your board.

- target_board_addr:  (*ip_address*, *port*)
  - This is the IP address that you would like to set the board to

- recv_addr:          (*ip_address*, *port*)
   - This is the IP address of the computer used to configure the IP address of the board
Set these parameters in the following code block.


In [ ]:
################
# Board Config #
################

board_model = "hdsoc2_evalr2" # Used for this example
workspace_folder = "board_workspace"


#############
# IP Config #
#############

default_board_addr = ("192.168.1.60", 4660)
target_board_addr = ("192.168.1.32", 4660)
recv_addr = ("192,168.1.198", 4660)

In [ ]:
##################
# Initialization #
##################

board = Board(board_model)

workspace_folder_path = os.path.join(os.getcwd(), workspace_folder)

if not os.path.exists(workspace_folder_path):
    os.mkdir(workspace_folder_path)


src_ip, src_port = target_board_addr
src_octets = get_connection_controller(board)._get_octets_or_raise(src_ip)

ethernet_src_config = {
    "eth_src_addr31_16": (src_octets[0] << 8) | src_octets[1],
    "eth_src_addr15_0": (src_octets[2] << 8) | src_octets[3],
    "eth_addr_sel": ("eth_dest_port", src_port)
}

board.start_server(os.path.join(os.getcwd(), 'board_workspace'))
board.connect_udp(default_board_addr, recv_addr)

with board:
    if startup_board(board, init_start=True):
        ControlRegisters(board).write_many(
            ethernet_src_config
        )

board.disconnect()
print(f"Disconnected from {board_model}")

## Testing Data Collection w/ New IP Address